### RB_OCS_BSS_CHARGE_COMPARE

- Выполняется сравнение OCS и BSS charge по кастомерам для коррекции OCS
- Строится график с различной аггрегацией по данным OCS

In [1]:
import pandas as pd
from pandas import Series, DataFrame
from os.path import join, normpath
from os import path
from collections import namedtuple
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import re

import roamability as rb

sponsor_object = namedtuple('Sponsor', ['name', 'mccmnc', 'sponsor_id'])

#######################################################################################
# Define variables here
#######################################################################################

rep_date_start = dt.date(2020,4,1) # Including
rep_date_end = dt.date(2020,4,30) # Including

downloads = 'C:/Users/balob/Downloads'

s1_sponsor = sponsor_object('S1', 42501, 1)
s2_sponsor = sponsor_object('S2', 26006, 2)
s4_sponsor = sponsor_object('S4', 45403, 4)
s5_sponsor = sponsor_object('S5', 51503, 5)

use_value = 'Revenue' # 'Cost', 'Revenue', 'Profit'

sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor]

file_name_ocs = f"ocs_reseller_charge_{rep_date_start.strftime('%Y_%m_%d')}_{rep_date_end.strftime('%Y_%m_%d')}.csv"

#######################################################################################

#######################################################################################

In [2]:
# Get data from OCS DB и Подготовка данных

sql_srt = """
SELECT
CONVERT(VARCHAR(10), u.USAGE_DATE, 103) AS Day
,s.DISPLAY_NAME AS Sponsor
,mm.NETWORK_ID AS VisitedNetworkId
,r.RESELLER_ID AS Customer
,SUM(uc.COST) AS Cost
,SUM(uc.CHARGE) AS Revenue
,ut.USAGE_TYPE AS UsageType
,MIN(uc.BALANCE) AS BALANCE
FROM [10028_usage].dbo.usage u
INNER JOIN [10028_usage].dbo.USAGE_CHARGES uc ON u.USAGE_ID = uc.USAGE_ID
INNER JOIN [10028].dbo.USAGE_TYPE ut ON u.USAGE_TYPE = ut.USAGE_TYPE_ID
INNER JOIN [10028].dbo.RESELLERS r ON uc.ACCOUNT_ID = r.ACCOUNT_ID
INNER JOIN [10028].dbo.RDB_NETWORK_IMSI_PREFIXES mm ON u.MCC = mm.MCC AND u.MNC = mm.MNC
INNER JOIN [10028].dbo.SPONSORS s ON u.GUIDING_INFO LIKE CONCAT(s.IMSI_PREFIX, '%')
WHERE
uc.CHARGE_ENTITY_TYPE = 'Reseller'
AND u.usage_date BETWEEN '{start_date}' AND '{end_date}'
GROUP BY
CONVERT(VARCHAR(10), u.USAGE_DATE, 103)
,ut.USAGE_TYPE
,r.RESELLER_ID
,r.RESELLER_NAME
,mm.NETWORK_ID
,s.DISPLAY_NAME""".format(start_date = rep_date_start.strftime('%Y-%m-%d %H:%M:%S'),
                          end_date = (rep_date_end + pd.Timedelta('1D')).strftime('%Y-%m-%d %H:%M:%S'))

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_ocs_source = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    
df_ocs_source.replace({'Data':'DATA','Incoming Text':'MTSMS','Outgoing Call':'MOC','Outgoing Text':'MOSMS'}, inplace=True)
    
# Подготовка данных

df_ocs_processed = df_ocs_source.copy()

# Подготовка колонки Profit
df_ocs_processed['Profit'] = df_ocs_processed['Revenue'] - df_ocs_processed['Cost']

# Подготовка широкого DF с заданием используемого значения (Cost, Revenue, Profit)
df_ocs_processed = df_ocs_processed.pivot_table(values=use_value,
                                                index=['Day','Sponsor','VisitedNetworkId','Customer'],
                                                columns='UsageType', aggfunc=np.sum).reset_index()

# Обработка значения даты с добавлением дополнительных полей даты
df_ocs_processed.Day = pd.to_datetime(df_ocs_processed.Day, format='%d/%m/%Y')
df_ocs_processed['Month'] = df_ocs_processed.Day.dt.strftime('%Y-%m')

# Добавление поля TOTAL и выборка используемых в дальнейшем полей
df_ocs_processed['TOTAL'] = df_ocs_processed[['MOC','MOSMS','MTSMS','DATA']].sum(axis=1)
df_ocs_processed['MTC'] = 0
usage_type_names = ['MOC','MTC','MOSMS','MTSMS','DATA', 'TOTAL']
df_ocs_processed[usage_type_names] = df_ocs_processed[usage_type_names].astype('float64')
df_ocs_processed = df_ocs_processed[
    ['Day','Month','Sponsor','VisitedNetworkId','Customer','MOC','MTC','MOSMS','MTSMS','DATA', 'TOTAL']].fillna(0)

df_ocs_processed.to_csv(join(downloads, file_name_ocs), index=False)

df_ocs_processed.head(3)

UsageType,Day,Month,Sponsor,VisitedNetworkId,Customer,MOC,MTC,MOSMS,MTSMS,DATA,TOTAL
0,2020-04-01,2020-04,S1,12,172844,0.115,0.0,0.012,0.0,0.00813,0.13513
1,2020-04-01,2020-04,S1,27,172760,0.000,0.0,0.000,0.0,0.01753,0.01753
2,2020-04-01,2020-04,S1,33,172844,0.000,0.0,0.000,0.0,0.00000,0.00000


In [3]:
# Получение данных из BSS и Подготовка данных

sql_srt="""
SELECT * FROM
(
SELECT
Day,SponsorPrefix AS Sponsor,VisitedNetworkId,Customer,Cost,Revenue
,'DATA' AS UsageType
FROM aggreg.GPRS_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
UNION ALL
SELECT
Day,SponsorPrefix AS Sponsor,VisitedNetworkId,Customer,Cost,Revenue
,CASE WHEN Sms = 0 THEN 'MOC' WHEN Sms = 1 THEN 'MOSMS' END AS UsageType
FROM aggreg.MOC_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
UNION ALL
SELECT
Day,SponsorPrefix AS Sponsor,VisitedNetworkId,Customer,Cost,Revenue
,CASE WHEN Sms = 0 THEN 'MTC' WHEN Sms = 1 THEN 'MTSMS' END AS UsageType
FROM aggreg.MTC_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
) t
""".format(start_date = rep_date_start.strftime('%Y%m%d'), end_date = rep_date_end.strftime('%Y%m%d'))

with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df_bss_source = pd.read_sql_query(sql_srt, cnxn)
# df_bss_source.to_csv(join(downloads, output_bss_usage_file), index=False)

# Подготовка данных и объединение со справочниками

df_bss_processed = df_bss_source.copy()

# Подготовка колонки Profit
df_bss_processed['Profit'] = df_bss_processed['Revenue'] - df_bss_processed['Cost']

# Замена префикса Спонсора на символьное обозначение? S1, S2...
sponsor_dict = dict((sponsor.mccmnc, sponsor.name) for sponsor in sponsors)
df_bss_processed.Sponsor.replace(sponsor_dict, inplace=True)

# Подготовка широкого DF с заданием используемого значения (Cost, Revenue, Profit)
df_bss_processed = df_bss_processed.pivot_table(values=use_value,
                                                index=['Day','Sponsor','VisitedNetworkId','Customer'],
                                                columns='UsageType', aggfunc=np.sum).reset_index()

# Обработка значения даты с добавлением дополнительных полей даты
df_bss_processed.Day = pd.to_datetime(df_bss_processed.Day, format='%Y%m%d')
df_bss_processed['Month'] = df_bss_processed.Day.dt.strftime('%Y-%m')

# Добавление поля TOTAL и выборка используемых в дальнейшем полей
df_bss_processed['TOTAL'] = df_bss_processed[['MOC','MTC','MOSMS','MTSMS','DATA']].sum(axis=1)
df_bss_processed = df_bss_processed[
    ['Day','Month','Sponsor','VisitedNetworkId','Customer','MOC','MTC','MOSMS','MTSMS','DATA', 'TOTAL']].fillna(0)

df_bss_processed.head(3)

UsageType,Day,Month,Sponsor,VisitedNetworkId,Customer,MOC,MTC,MOSMS,MTSMS,DATA,TOTAL
0,2020-04-01,2020-04,S1,12,172716,0.000,0.0,0.000,0.0,1.066840,1.066840
1,2020-04-01,2020-04,S1,12,172844,0.114,0.0,0.024,0.0,0.008348,0.146348
2,2020-04-01,2020-04,S1,27,172717,0.000,0.0,0.000,0.0,0.263875,0.263875


In [4]:
# Загрузка справочников из OCS

# sql_srt='SELECT network_id AS VisitedNetworkId, MAX(tadig_code) AS Tadig FROM RDB_TADIG_CODES GROUP BY network_id;'

sql_srt="""SELECT t.network_id AS VisitedNetworkId, MAX(c.country_name) AS Country, MAX(t.tadig_code) AS Tadig
FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
GROUP BY t.network_id;"""

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
display(df_tadig.head(3))

sql_srt='SELECT reseller_id AS Customer, reseller_name AS CustomerName FROM RESELLERS;'
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_customer = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    
df_customer.replace({'STI - Telinta':'STI'}, inplace=True)

display(df_customer.head(3))

sql_srt='''
SELECT
r.RESELLER_ID AS Customer
,MAX(uc.balance) AS START_BALANCE
FROM [10028_usage].dbo.USAGE_CHARGES uc
LEFT JOIN [10028].dbo.RESELLERS r ON uc.account_id = r.account_id
WHERE r.RESELLER_ID IS NOT NULL
AND uc.charge_date BETWEEN '{start_date}' AND '{end_date}'
GROUP BY r.RESELLER_ID'''.format(start_date = rep_date_start.strftime('%Y-%m-%d %H:%M:%S'),
                          end_date = (rep_date_start + pd.Timedelta('1D')).strftime('%Y-%m-%d %H:%M:%S'))

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_balance = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    
display(df_balance.head(3))

,VisitedNetworkId,Country,Tadig
0,4,Afghanistan,AFGAW
1,5,Afghanistan,AFGAR
2,6,Afghanistan,AFGEA


,Customer,CustomerName
0,172711,Mondicon
1,172716,Cloud 9 TAP
2,172717,KnowRoaming IMSI-RANGE TAP


,Customer,START_BALANCE
0,172875,5225.58688
1,172854,1459.22914
2,172876,4645.67418


In [5]:
# Агрегация по времени

customers_list = ['Dialoq - Genesisclub RT', 'Tottolli-Tech-Mobil', 'STI', 'Drimsim', 'X2One', 'Netmore DMI RT', 'Clay RT']

aggregation_time = ['Month'] # Month, Day
aggregation_list = ['Customer'] # 'Sponsor','VisitedNetworkId','Customer'
usage_type = ['TOTAL']

# Сравнение

df_temp = pd.merge(
    df_bss_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    df_ocs_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    how='outer', on = aggregation_list + aggregation_time, suffixes=['_BSS','_OCS']
).dropna()

cols = (aggregation_time + ['CustomerName'] + aggregation_list + ['TOTAL_BSS', 'TOTAL_OCS', 'START_BALANCE'])
cols.remove('Customer')

df_temp = pd.merge(df_temp, df_customer, how='left', on='Customer')
df_temp = pd.merge(df_temp, df_balance, how='left', on='Customer')[cols]

df_temp['DIFF'] = df_temp.TOTAL_OCS - df_temp.TOTAL_BSS
df_temp.loc[df_temp.CustomerName.isin(customers_list)]

,Month,CustomerName,TOTAL_BSS,TOTAL_OCS,START_BALANCE,DIFF
0,2020-04,Netmore DMI RT,1916.003044,1935.97863,5492.25498,19.975586
4,2020-04,Dialoq - Genesisclub RT,1354.005911,1979.60791,4577.55544,625.601999
9,2020-04,Tottolli-Tech-Mobil,20897.732402,22374.98322,2982.10642,1477.250818
10,2020-04,STI,6520.537498,6485.67040,1459.22914,-34.867098
13,2020-04,X2One,1780.624465,1879.78938,1290.08967,99.164915
14,2020-04,Drimsim,913.723190,1018.75452,13699.78079,105.031330


In [20]:
# Correction May 01, 20 - (overcharge)
# Invoice #1827
payments = 2403.60
5000 - 4288.32959

711.6704099999997

In [8]:
# Детализация полная

aggregation_time = ['Month'] # Month, Day
aggregation_list = ['Customer','Sponsor','VisitedNetworkId']
usage_type = ['MOC','MTC','MOSMS','MTSMS','DATA']
customer = ['Dialoq - Genesisclub RT']

output_diff_file = f"bss_ocs_diff_full_{rep_date_start.strftime('%Y%m%d')}_{rep_date_end.strftime('%Y%m%d')}.xlsx"

# Сравнение

df_temp = pd.merge(
    df_bss_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    df_ocs_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    how='outer', on = aggregation_list + aggregation_time, suffixes=['_BSS','_OCS']
).fillna(0)

df_temp = pd.merge(df_temp, df_customer, how='left', on='Customer')
df_temp = pd.merge(df_temp, df_tadig, how='left', on='VisitedNetworkId')
df_temp.drop(['VisitedNetworkId','Customer'], axis=1, inplace=True)

df_temp = df_temp.loc[df_temp.CustomerName.isin(customer)]

cols = aggregation_time + ['Country','Sponsor'] + ['Tadig','CustomerName',
                                              'MOC_BSS','MOC_OCS','MTC_BSS','MTC_OCS','MOSMS_BSS','MOSMS_OCS',
                                              'MTSMS_BSS','MTSMS_OCS','DATA_BSS','DATA_OCS']

display(df_temp[cols].head())
df_temp[cols].to_excel(join(downloads, output_diff_file), index=False)

,Month,Country,Sponsor,Tadig,CustomerName,MOC_BSS,MOC_OCS,MTC_BSS,MTC_OCS,MOSMS_BSS,MOSMS_OCS,MTSMS_BSS,MTSMS_OCS,DATA_BSS,DATA_OCS
1156,2020-03,Albania,S1,ALBVF,Dialoq - Genesisclub RT,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,4.456332,4.61810
1157,2020-03,Argentina,S1,ARGTM,Dialoq - Genesisclub RT,0.0,0.0,0.0,0.0,0.180,0.180,0.0,0.0,61.860109,62.68866
1158,2020-03,Armenia,S1,ARM01,Dialoq - Genesisclub RT,0.0,0.0,0.0,0.0,3.700,3.700,0.0,0.0,10.926650,11.04568
1159,2020-03,Australia,S1,AUSOP,Dialoq - Genesisclub RT,0.0,0.0,0.0,0.0,0.024,0.024,0.0,0.0,30.243551,30.99616
1160,2020-03,Austria,S1,AUTPT,Dialoq - Genesisclub RT,0.0,0.0,0.0,0.0,2.490,2.490,0.0,0.0,200.398040,209.91961


In [13]:
# Детализация с подкраской

aggregation_time = ['Month'] # Month, Day
aggregation_list = ['Customer','Sponsor','VisitedNetworkId'] # 'Sponsor','VisitedNetworkId','Customer'
usage_type = ['MOC','MTC','MOSMS','MTSMS','DATA']
customer = ['Dialoq - Genesisclub RT','Joy_telecom','Tottolli-Tech-Mobil']
rel_tolerance = 0.1 # 1 - 100% от Chgarge Example: Cost = 10 USD, Charge < 10 USD

output_diff_file = f"bss_ocs_diff_colored_{rep_date_start.strftime('%Y%m%d')}_{rep_date_end.strftime('%Y%m%d')}.xlsx"

# aggregation_list.extend(aggregation_time)

# Сравнение

df_temp = pd.merge(
    df_bss_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    df_ocs_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    how='outer', on = aggregation_list + aggregation_time, suffixes=['_BSS','_OCS']
).fillna(0)

df_temp = pd.merge(df_temp, df_customer, how='left', on='Customer')

df_temp = df_temp.loc[df_temp.CustomerName.isin(customer)]

cols = aggregation_time + ['Country'] + aggregation_list + ['Tadig','CustomerName',
                                              'MOC_BSS','MOC_OCS','MTC_BSS','MTC_OCS','MOSMS_BSS','MOSMS_OCS',
                                              'MTSMS_BSS','MTSMS_OCS','DATA_BSS','DATA_OCS']

if 'Customer' in cols:
    cols.remove('Customer')
if 'VisitedNetworkId' in cols:
    cols.remove('VisitedNetworkId')

df_temp = pd.merge(df_temp, df_tadig, how='left', on='VisitedNetworkId')[cols]

df_diff = df_temp[
    ~np.isclose(df_temp.MOC_BSS, df_temp.MOC_BSS, rtol=rel_tolerance, equal_nan=True) |\
    ~np.isclose(df_temp.MTC_BSS, df_temp.MTC_BSS, rtol=rel_tolerance, equal_nan=True) |\
    ~np.isclose(df_temp.MOSMS_BSS, df_temp.MOSMS_OCS, rtol=rel_tolerance, equal_nan=True) |\
    ~np.isclose(df_temp.MTSMS_BSS, df_temp.MTSMS_OCS, rtol=rel_tolerance, equal_nan=True) |\
    ~np.isclose(df_temp.DATA_BSS, df_temp.DATA_OCS, rtol=rel_tolerance, equal_nan=True)
]

def colorize(s):
    color_list = []
    for tariff_type in [['MOC_BSS','MOC_BSS'], ['MTC_BSS','MTC_BSS'],
                        ['MOSMS_BSS','MOSMS_OCS'], ['MTSMS_BSS','MTSMS_OCS'], ['DATA_BSS','DATA_OCS']]:
        if np.isclose(s[tariff_type[0]], s[tariff_type[1]], rtol=rel_tolerance, equal_nan=True):
            color_list.extend(['', ''])
        elif s[tariff_type[0]] > s[tariff_type[1]]:
            color_list.extend(['background-color: yellow', 'background-color: red'])
        else:
            color_list.extend(['background-color: yellow', 'background-color: #81EF21'])
    return color_list

df_diff_style = df_diff.style.apply(colorize,
                            subset=['MOC_BSS','MOC_OCS','MTC_BSS','MTC_OCS','MOSMS_BSS','MOSMS_OCS',
                          'MTSMS_BSS','MTSMS_OCS','DATA_BSS','DATA_OCS'], axis=1).hide_index()

# display(df_diff_style)
df_diff_style.to_excel(join(downloads, output_diff_file), index=False)

Проблемы
1. Не учитываются МО СМС для Стиринга в OCS.
2. Спонсоры предоставляют неправильные тарифы.
3. Проблема с неправильным определением страны пребывания в OCS